In [1]:
import qval as qv
import numpy as np
import random

In [2]:

bits = 8

def quantize_matrix(matrix, n):
    rows, cols = matrix.shape

    qmatrix = [[0 for _ in range(rows)] for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            qmatrix[i][j] = qv.quantize(matrix[i][j], n)

    return np.array(qmatrix)

def dequantize_matrix(qmatrix, n):
    rows, cols = qmatrix.shape

    matrix = [[0 for _ in range(rows)] for _ in range(cols)]

    for i in range(rows):
        for j in range(cols):
            matrix[i][j] = qv.dequantize(qmatrix[i][j], n)

    return np.array(matrix)



In [16]:
def qMatMul(m1, m2, n):
    rows = m1.shape[0]
    cols = m2.shape[1]
    muls = m2.shape[0]

    m3 = np.array([[qv.qZERO(n) for _ in range(cols)] for _ in range(rows)], dtype=object)

    for i in range(rows):
        for j in range(cols):
            sum_ij = qv.qZERO(n)
            for k in range(muls):
                prod = qv.qMul(m1[i][k], m2[k][j])
                prev_sum = sum_ij
                sum_ij = qv.qAdd(sum_ij, qv.qfit(prod, len(sum_ij)))
                # print(f"Adding: {qv.dequantize(prev_sum)} + {qv.dequantize(prod)}, i: {qv.dequantize(prev_sum) + qv.dequantize(prod)}, q:{qv.dequantize(sum_ij)}")
            m3[i][j] = qv.qfit(sum_ij, n)

    return m3



def MatMul(m1, m2):
    rows = m1.shape[0]
    cols = m2.shape[1]
    muls = m2.shape[0]

    m3 = np.array([[0 for _ in range(cols)] for _ in range(rows)], dtype=object)

    for i in range(rows):
        for j in range(cols):
            sum_ij = 0
            for k in range(muls):
                
                prod = m1[i][k] * m2[k][j]
                sum_ij += prod
            
            m3[i][j] = sum_ij

    return m3

def matrix_mean_error(matrix, qmatrix):
    error_sum = 0
    for i in range(3):
        for j in range(3):
            error_sum += matrix[i][j] - qmatrix[i][j]

    return error_sum / 9

In [36]:
def mean_error():
    error = 0
    tests = 10000
    for i in range(tests):
        mat1 = np.random.uniform(-1, 1, (3, 3))
        mat2 = np.random.uniform(-1, 1, (3, 3))
    
        qmat1 = quantize_matrix(mat1, bits)
        qmat2 = quantize_matrix(mat2, bits)
    
        qmultmat = qMatMul(qmat1, qmat2, bits)
        qmultmat = dequantize_matrix(qmultmat, bits)
        multmat = MatMul(mat1, mat2)
    
        error += matrix_mean_error(multmat, qmultmat)
    return error/tests

print(mean_error())


-0.08425431353842151


array([[-0.5280214738723044, 0.45960742546494526, 0.09170268026185055],
       [-0.42072707273493903, 0.21763617814710245, -0.3377369714648636],
       [-0.7308341385395363, 0.5331974667929116, 0.13965635628030512]],
      dtype=object)

In [6]:

print(qv.qMax_i(bits))
a = random.uniform(0, 1)
b = random.uniform(0, 1)
print(f"a: {a}, b: {b}, a+b: {a+b}")
qa = qv.quantize(a, bits)
qb = qv.quantize(b, bits)
print(f"qa: {qa}, qb: {qb}, qa+qb: {qv.qAdd(qa, qb)}")
print(f"qai: {qv.qToInt(qa)}, qbi: {qv.qToInt(qb)}, (qa+qb)i: {qv.qToInt(qv.qAdd(qa, qb))}")
print(f"dqa: {qv.dequantize(qa)}, dqb: {qv.dequantize(qb)}, d(qa+qb): {qv.dequantize(qv.qAdd(qa, qb), bits)}")
print(qv.dequantize(qv.qFromInt(qv.qToInt(qv.qAdd(qa, qb)), bits), bits))

127
a: 0.9762113479316431, b: 0.5692960697102762, a+b: 1.5455074176419195
qa: 01111100, qb: 01001000, qa+qb: 01111111
qai: 124, qbi: 72, (qa+qb)i: 127
dqa: 0.9763779527559056, dqb: 0.5669291338582677, d(qa+qb): 1.0
1.0


In [7]:
bits = 8
print(qv.qMax_i(bits))
print(f"a: {a}, b: {b}, a*b: {a*b}")
qa = qv.quantize(a, bits)
qb = qv.quantize(b, bits)
print(f"qa: {qa}, qb: {qb}, qa*qb: {qv.qMul(qa, qb)}")
print(f"qai: {qv.qToInt(qa)}, qbi: {qv.qToInt(qb)}, (qa*qb)i: {qv.qToInt(qv.qMul(qa, qb))}")
print(f"dqa: {qv.dequantize(qa)}, dqb: {qv.dequantize(qb)}, d(qa*qb): {qv.dequantize(qv.qMul(qa, qb), None)}")
print(qv.dequantize(qv.qFromInt(qv.qToInt(qv.qMul(qa, qb)), bits), bits))

127
a: 0.9762113479316431, b: 0.5692960697102762, a*b: 0.5557532835840554
qa: 01111100, qb: 01001000, qa*qb: 01000101
qai: 124, qbi: 72, (qa*qb)i: 69
dqa: 0.9763779527559056, dqb: 0.5669291338582677, d(qa*qb): 0.5433070866141733
0.5433070866141733
